<a href="https://colab.research.google.com/github/yasohasakii/binary_categorical/blob/categorical_version/resnet_binary_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# git clone repository
!git clone https://github.com/yasohasakii/binary_categorical.git
!mv ./binary_categorical/* ./
!rm -r ./binary_categorical ./sample_data
!pip install tensorflow-gpu
!nvidia-smi

Cloning into 'binary_categorical'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 22693 (delta 7), reused 0 (delta 0), pack-reused 22679
Receiving objects: 100% (22693/22693), 1.01 GiB | 37.41 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Checking out files: 100% (22661/22661), done.
     |████████████████████████████████| 377.0MB 79kB/s 
Mon Sep  2 01:32:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000

In [2]:
# import modual
from __future__ import print_function
import numpy as np
import os, glob, time
from keras.layers import Input
from keras.models import Model
from keras.preprocessing import image
from keras import regularizers, optimizers
from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.applications.resnet50 import ResNet50
os.environ['CUDA_VISIBLE_DEVICES']='0'

np.random.seed(1337)

Using TensorFlow backend.


In [0]:
# assert parameters
batch_size = 32
EPOCHS=20

train_count = len(glob.glob('./PetImages/train/*/*.jpg'))
valid_count = len(glob.glob('./PetImages/test/*/*.jpg'))

In [0]:
# def build_model
def build_model(trainable=True):
    input = Input(shape = (224,224,3),name="kfb_image")
    base_model = ResNet50(include_top=False, weights='imagenet', input_tensor=input) 
    x = base_model.output
    # for layer in base_model.layers:
    #     layer.trainable=False
    x = GlobalAveragePooling2D()(x)
    if trainable:
        x = Dropout(0.5,name="dropout_1")(x)
    prediction = Dense(2, activation='softmax',name = "dense_3")(x)
    model = Model(inputs=base_model.input, outputs=prediction)
    # model.summary()
    return model

In [0]:
# def train
def train(train_count,valid_count):
    train_datagen = image.ImageDataGenerator(
        rescale = 1./255, 
        rotation_range=40,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest')
    valid_datagen = image.ImageDataGenerator(
        rescale = 1./255)

    train_generator = train_datagen.flow_from_directory(
        './PetImages/train/',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
    validation_generator = valid_datagen.flow_from_directory(
        './PetImages/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')

    model = build_model()
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=1e-2,decay =1e-3), metrics=["accuracy"])
    models_save_path = "./models"
    if not os.path.exists(models_save_path):
        os.makedirs(models_save_path)

    checkpoint = ModelCheckpoint(filepath=os.path.join(models_save_path, 'resnet-{epoch:02d}-{val_acc:.4f}.h5'),
                                 monitor='val_acc',
                                 mode='max',
                                 save_best_only=True,
                                 save_weights_only=True)
    print("Train files: {}, valid files: {}".format(train_count,valid_count))
    print('-----------Start training-----------')
    start = time.time()
    history = model.fit_generator(train_generator,
                        steps_per_epoch=train_count // batch_size,
                        epochs=EPOCHS,
                        initial_epoch=0,
                        validation_data=validation_generator,
                        validation_steps=valid_count // batch_size,
                        callbacks=[checkpoint],
                        use_multiprocessing=False)
    end = time.time()
    print("train finished, cost time = {} hours".format(round((end - start) / 3600.0,3)))
    return history

In [0]:
# def plot modual
def plot_train_history(history):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(20, 10))
    plt.subplot(1,2,1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.savefig("history.png")

In [0]:
# def test
def test(model_path,image_dir):
    model = build_model(trainable=False)
    model.load_weights(model_path, by_name=True)
    for layer in model.layers:
        layer.trainable=False
    images = glob.glob(image_dir+"/*.jpg")
    images.sort()
    cat=0.0
    dog=0.0
    for imgs in images:
        im = image.load_img(imgs, target_size=(224, 224))
        im = image.img_to_array(im)* 1. / 255
        im = np.expand_dims(im, axis=0)
        out = model.predict(im)[0]
        preds = np.argmax(out,axis=0)
        if preds>=0.5:
            dog += 1
        else:
            cat += 1
    print(dog+cat)
    if dog>cat:
        print("{}: dog, score{}, inage nums {}".format(os.path.basename(image_dir),dog/(dog+cat),len(images)))
    else:
        print("{}: cat, score{}, image nums {}".format(os.path.basename(image_dir),cat/(dog+cat),len(images)))

In [0]:
if __name__=="__main__":
    history = train(train_count,valid_count)

Found 15894 images belonging to 2 classes.
Found 6754 images belonging to 2 classes.


/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Train files: 15894, valid files: 6754
-----------Start training-----------
Epoch 1/20
132/496 [======>.......................] - ETA: 5:50 - loss: 0.2919 - acc: 0.8710

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 209715200 bytes but only got 0. Skipping tag 48
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 52428800 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6468665344 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1050744 bytes but only got 7027. Skipping tag 48
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expe

156/496 [========>.....................] - ETA: 5:19 - loss: 0.2716 - acc: 0.8812

...

363/496 [====================>.........] - ETA: 1:55 - loss: 0.1975 - acc: 0.9174

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


376/496 [=====================>........] - ETA: 1:44 - loss: 0.1958 - acc: 0.9180

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:725: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but 

496/496 [==============================] - 483s 974ms/step - loss: 0.1793 - acc: 0.9259 - val_loss: 0.0758 - val_acc: 0.9679
Epoch 2/20
496/496 [==============================] - 461s 930ms/step - loss: 0.0857 - acc: 0.9664 - val_loss: 0.0470 - val_acc: 0.9819
Epoch 3/20
496/496 [==============================] - 464s 935ms/step - loss: 0.0743 - acc: 0.9717 - val_loss: 0.0521 - val_acc: 0.9811
Epoch 4/20
496/496 [==============================] - 464s 935ms/step - loss: 0.0583 - acc: 0.9761 - val_loss: 0.0479 - val_acc: 0.9827
Epoch 5/20
496/496 [==============================] - 465s 937ms/step - loss: 0.0531 - acc: 0.9797 - val_loss: 0.0373 - val_acc: 0.9853
Epoch 6/20
495/496 [============================>.] - ETA: 0s - loss: 0.0449 - acc: 0.9827

In [33]:
plot_train_history(history)

NameError: ignored

In [36]:
test('./models/resnet-02-0.9759.h5','./PetImages/test/Dog')

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


3379.0
Dog: dog, score0.9709973364900858, inage nums 3379


In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving resnet-04-0.9722.h5 to resnet-04-0.9722.h5
User uploaded file "resnet-04-0.9722.h5" with length 96946840 bytes


In [23]:
valid_datagen = image.ImageDataGenerator(
        rescale = 1./255)
validation_generator = valid_datagen.flow_from_directory(
        './PetImages/test',
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
files = validation_generator.filenames
classes = validation_generator.classes
for i in range(2):
    print(validation_generator.next())
print(validation_generator.class_indices)

Found 6754 images belonging to 2 classes.
(array([[[[0.627451  , 0.03529412, 0.        ],
         [0.6431373 , 0.05490196, 0.        ],
         [0.6509804 , 0.04705883, 0.        ],
         ...,
         [1.        , 0.20000002, 0.15686275],
         [1.        , 0.20784315, 0.15294118],
         [1.        , 0.21960786, 0.14509805]],

        [[0.63529414, 0.04313726, 0.01568628],
         [0.64705884, 0.05490196, 0.01960784],
         [0.6627451 , 0.04705883, 0.01960784],
         ...,
         [1.        , 0.23137257, 0.16862746],
         [1.        , 0.227451  , 0.16078432],
         [1.        , 0.21960786, 0.12941177]],

        [[0.6156863 , 0.03921569, 0.01176471],
         [0.627451  , 0.04313726, 0.01960784],
         [0.6431373 , 0.03529412, 0.01568628],
         ...,
         [1.        , 0.227451  , 0.15294118],
         [1.        , 0.22352943, 0.14901961],
         [1.        , 0.21568629, 0.15294118]],

        ...,

        [[0.7176471 , 0.04705883, 0.00784314],
  